# Librerias

In [16]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from scipy import signal
import os
%matplotlib widget

# Declaracion de constantes 

In [17]:
F_MIN = 5
F_MAX = 40
F_MUESTREO = 128
RUTA_DATOS_PROCESADOS = '../DatosProcesados/Frecuencia'
CARPETAS = ['Rojo', 'Azul', 'Verde', 'Morado', 'Baseline']

# Declaracion de Funciones 

In [50]:
def plot_datos(datos):
    
    plt.figure()
    plt.plot(datos[:, 0], label = 'EEG.O1')
    plt.plot(datos[:, 1], label = 'EEG.O2')
    plt.legend()
    plt.title('Raw Data')
    plt.xlabel('Tiempo')
    plt.ylabel('uV')
    
def plot_psd_semilog(datos):
    
    f1, psd_O1 = signal.periodogram(datos[:, 0], F_MUESTREO)
    f2, psd_O2 = signal.periodogram(datos[:, 1], F_MUESTREO)
    plt.semilogy(f1, psd_O1, label = 'EEG.O1')
    plt.semilogy(f1, psd_O2, label = 'EEG.O2')
    plt.ylim([1e-7, 1e2])
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.legend()

    
def plot_psd(datos):
    
    plt.figure()
    f1, psd_O1 = signal.periodogram(datos[:, 0], F_MUESTREO)
    f2, psd_O2 = signal.periodogram(datos[:, 1], F_MUESTREO)
    plt.plot(f1, psd_O1, label = 'EEG.O1')
    plt.plot(f1, psd_O2, label = 'EEG.O2')
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('PSD')
    plt.legend()
    print(psd_O1.mean())
    print(psd_O2.mean())
    print(max(psd_O1))
    print(max(psd_O2))

def filtro_pasa_banda(datos, f_min, f_max, f_muestreo, orden=5):
    nyq = 0.5 * f_muestreo
    f_min = f_min / nyq
    f_max = f_max / nyq
    b, a = signal.butter(orden, [f_min, f_max], btype='band')
    print(b, a)
    y = signal.lfilter(b, a, datos)
    return y



## Graficas de Datos (Sin filtro)

In [60]:
from scipy import fftpack 

for carpeta in CARPETAS:
    archivos = [e for e in os.listdir(RUTA_DATOS_PROCESADOS+'/'+carpeta) if e.endswith('.csv') ]
    
    for indice, archivo in enumerate(archivos):
        ruta_origen = f'{RUTA_DATOS_PROCESADOS}/{carpeta}/{archivo}'
        print(archivo)
        df = pd.read_csv(ruta_origen)
        etiqueta = df[['MarkerValueInt']]
        datos = df[['EEG.O1', 'EEG.O2']].to_numpy()
        o1 = fftpack.hilbert(datos[:, 0])
        o1 = pd.DataFrame(data=o1, columns=["EEG.O1"])
        o2 = fftpack.hilbert(datos[:, 0])
        o2 = pd.DataFrame(data=o2, columns=["EEG.O2"])
        archivo = pd.concat([o1, o2, etiqueta], axis=1).to_csv(f'{RUTA_DATOS_PROCESADOS}/{carpeta}/hilbert{indice}.csv', index = False)

muestra11.csv
